# MT-bench Evaluation System Demo\n\nThis notebook demonstrates the MT-bench evaluation system by cloning the repository and running the demo.

In [ ]:
# Clone the mt_bench repository\n!rm -rf mt_bench\n!git clone https://github.com/allenlu2009/mt_bench.git\n%cd mt_bench\n%pwd

In [ ]:
# Install dependencies\n!pip install -r requirements.txt

In [ ]:
# Run the demo\n!python demo_cli.py